<hr style="border:3px solid black"></hr>
Contact : dhilip@iitrpr.ac.in <br>

    - Apoorv Kushwaha & Dr. T.J. Dhilip Kumar
        Quantum Dynamics Lab (410), IIT Ropar.
***
# Psi4 template for calculating rigid rotor-atom collision PES:

Link :  https://psicode.org/

<hr style="border:3px solid black"></hr>


<hr style="border:2px solid black"></hr>

### *Import libraries ---> Create required folders  ---> Generate input files*

    - Example Input  : Make Required changes in the cell as needed. [green]
    - Example Output : Describes the output of the cell. [indigo]

<hr style="border:2px solid black"></hr>

In [1]:
# Import required libraries
import math
import numpy as np
import pandas as pd
import os
from scipy.special import legendre
import matplotlib.pyplot as plt
import psi4
from tqdm.notebook import tqdm

In [2]:
# creating required folders
input_dir = os.getcwd()+'/'

out_plots = input_dir + 'plots/'      # directory for plots
if not os.path.exists(out_plots):
    os.makedirs(out_plots)
out_data = input_dir + 'data/'   # directory for psi4 data
if not os.path.exists(out_data):
    os.makedirs(out_data)

psi4_data = input_dir + 'psi4_data/'   # directory for psi4 data
if not os.path.exists(psi4_data):
    os.makedirs(psi4_data)
pes_data = psi4_data + 'C2H2/'         # directory for PES
if not os.path.exists(pes_data):
    os.makedirs(pes_data)
    
scratch_dir = psi4_data + 'C2H2_scratch/'      # scratch directory for 
psi4_io = psi4.core.IOManager.shared_object()
psi4_io.set_default_path(scratch_dir)
if not os.path.exists(scratch_dir):
    os.makedirs(scratch_dir)

<hr style="border:2px solid black"></hr>

### Creating matrix containing R, theta coordinates
***
    
<span style="color:green"><u> Example Input _[via code]_ </u></span>
   
    r1 = np.arange(2.0,20.1,0.1)                 # Step size to create radial coordinate    (R)
    for i_gamma in tqdm(range (0,91,15)):        # Step size to create angular coordinate  (Phi)
***
<span style="color:indigo"><u> Example Output</u></span>
    
    Radial  coordinates go from 2.0 to 20.0  Angstroms with step size of 0.1 Angstrom.
    Angular coordinates go from   0 to 90    Degrees   with step size of 15  degrees.
    Coordinates are saved to data/r_gam.dat file.
    
    Live: tqdm shows the remaining time!
<hr style="border:2px solid black"></hr>

In [3]:
A = np.ndarray(shape=(1,4)) # garbage array initialization
R1 = np.arange(2,9.0,0.1) 
R2 = np.arange(9.0,14.0,0.5)
R3 = np.arange(14,21.0,1.0)
R = np.concatenate((R1,R2,R3),axis=None)
r_n = len(R) # saving number of R data points
r4 = np.atleast_2d(R).T   # creating r column vector
for i_phi in tqdm(range (0,91,30)):
    for j_theta2 in range (0,91,30):
        for k_theta1 in range (0,181,30):
            b = np.array([i_phi,j_theta2,k_theta1])
            c = np.tile(b,(r_n,1))  # creating angles as columns
            d = np.c_[ r4, c ]        # joining r and columns
            A = np.vstack([A, d]) # repeating for different geoms and joining
A = np.delete(A, 0, 0) # deleting first row (empty)
np.savetxt(out_data + "4D_coordinates.dat", A, fmt='%.1f\t%d\t%d\t%d')
print("Number of data points in the saved file : ",len(A))

  0%|          | 0/4 [00:00<?, ?it/s]

Number of data points in the saved file :  9744


<hr style="border:2px solid black"></hr>

### Creating matrix containing R, theta coordinates
***
    
<span style="color:green"><u> Example Input _[via code]_ </u></span>
    
    rcc : half length of C-C molecule (center of mass position)
    psi4.set_memory('500 MB') : Set memory required
    set_num_threads(4)        : Set number of threats

    Input gemoetry and calculation options can be obtained by following link:
https://psicode.org/psi4manual/1.4.0/psithoninput <br>
https://psicode.org/psi4manual/1.4.0/nbody

***
<span style="color:indigo"><u> Example Output</u></span>
    
    Energies are calculated and stored in ecp variable
    The output file for each calculation are saved in /psi4_data/C2He/{R}_{gamma}.out
    
    Live: tqdm shows the remaining time!
<hr style="border:2px solid black"></hr>

In [ ]:
r1 = 0.6215    # half of C-C distance
r3 = 0.3707    # half H-H distance               
ecp = {}
c2h2_geom = """
0 1
C          {:.6f}        {:.6f}        {:.6f} 
C          {:.6f}        {:.6f}        {:.6f} 
--
0 1
H         0.000000          {:.6f}        {:.6f}
H         0.000000          {:.6f}        {:.6f}
"""

psi4.set_memory('2 GB')
psi4.set_num_threads(2) 

for j in tqdm(range (len(A))):      # python loop fruns from last position to first position
    try:
        R = A[j,0]
        phi = A[j,1]
        theta2 = A[j,2]
        theta1 = A[j,3]
        
        # 3D projection for C2
        a1=math.sin(math.radians(theta1))*math.sin(math.radians(phi))
        a2=math.sin(math.radians(theta1))*math.cos(math.radians(phi))
        a3=math.cos(math.radians(theta1))
        
        # 2D projection for H2
        b1=math.sin(math.radians(theta2))
        b2=math.cos(math.radians(theta2))
        psi4.set_output_file(pes_data+'{:.2f}_{:d}_{:d}_{:d}.out'.format(R,int(phi),int(theta2),int(theta1)),
                             False) # sets output file location
        
        c2h2 = psi4.geometry(c2h2_geom.format(r1*a1,  r1*a2,  r1*a3,
                                              r1*a1, -r1*a2, -r1*a3,
                                              r3*b1,R+r3*b2,
                                              -r3*b1,R-r3*b2))
        with  open(pes_data+'{:.2f}_{:d}_{:d}_{:d}.inp'.format(R, int(phi),int(theta2),int(theta1)),'w') as myfile:
            myfile.write(c2h2_geom.format(r1*a1,  r1*a2,  r1*a3,
                                          r1*a1, -r1*a2, -r1*a3,
                                          r3*b1,R+r3*b2,
                                          -r3*b1,R-r3*b2))
        
        psi4.set_options({'reference': 'rhf','freeze_core': True})
        ecp[j] = psi4.energy('ccsd(t)/aug-cc-pvdz', bsse_type='cp', return_total_data=True, molecule=c2h2)
    except:
        pass


  Memory set to   1.863 GiB by Python driver.
  Threads set to 2 by Python driver.


  0%|          | 0/9744 [00:00<?, ?it/s]

In [5]:
ecp

{3: -64.76668006877065,
 4: -70.85223011144294,
 5: -72.52995999080368,
 6: -72.02906384055912,
 7: -68.71399414504013,
 10: -58.95786724252007,
 11: -70.16845191456598,
 12: -73.57567553639309,
 13: -75.12520990483473,
 14: -75.92400566908616,
 15: -76.35564755749526,
 16: -76.59454850315562,
 17: -76.72933016853358,
 18: -76.80623713460946,
 19: -76.84967112935401}

In [23]:
R = 200
gamma = 90
r1=math.sin(math.radians(gamma))*R
r2=math.cos(math.radians(gamma))*R
c2he = psi4.geometry(c2he_geom.format(-rcc,rcc,r1,r2))

psi4.set_options({'reference': 'rhf','freeze_core': True})
ecp_inf = psi4.energy('ccsd(t)/aug-cc-pvdz', bsse_type='cp', return_total_data=True, molecule=c2he)
ecp_inf

-78.62306058276586

In [24]:
data = np.array(list(ecp.items()))
arr = data[:,1]
reversed_arr = arr[::-1]
reversed_arr

array([-78.56108373, -78.57610346, -78.58826363, -78.59734457,
       -78.60413008, -78.60927398, -78.61317231, -78.61608935,
       -78.61823368, -78.61978108, -78.62087817, -78.62164322,
       -78.62216827, -78.62252286, -78.62275816, -78.62291116,
       -78.62300813, -78.62306746, -78.62310192, -78.62312015,
       -78.62312811, -78.62312975, -78.62312759, -78.62312342,
       -78.62311825, -78.62311276, -78.62310735, -78.62310225,
       -78.62309756, -78.62309333, -78.62308955, -78.6230862 ,
       -78.62308325, -78.62308066, -78.62307839, -78.62307639,
       -78.62307465, -78.62307313, -78.62307179, -78.62307062,
       -78.62306958, -78.62306869, -78.62306788, -78.62306717,
       -78.62306653, -78.62306596, -78.62306546, -78.62306501,
       -78.6230646 , -78.62306424, -78.62306392, -78.62306363,
       -78.62306337, -78.62306313, -78.62306292, -78.62306273,
       -78.62306256, -78.6230624 , -78.62306226, -78.62306213,
       -78.62306201, -78.6230619 , -78.6230618 , -78.62

In [26]:
final_data = np.c_[ A, reversed_arr ]
np.savetxt(out_data + "2D_PES_psi4.dat", final_data, delimiter=",",fmt='%.2f,%d,%.14f')
final_data

array([[  2.        ,   0.        , -78.56108373],
       [  2.1       ,   0.        , -78.57610346],
       [  2.2       ,   0.        , -78.58826363],
       ...,
       [ 18.        ,  90.        , -78.62306059],
       [ 19.        ,  90.        , -78.62306059],
       [ 20.        ,  90.        , -78.62306059]])

In [28]:
reversed_arr_cm = (reversed_arr-ecp_inf)*219474.6
final_data_cm = np.c_[ A, reversed_arr_cm ]
np.savetxt(out_data + "2D_PES_psi4_cm.dat", final_data_cm, delimiter=",",fmt='%.2f,%d,%.4f')
final_data_cm

array([[ 2.00000000e+00,  0.00000000e+00,  1.36023445e+04],
       [ 2.10000000e+00,  0.00000000e+00,  1.03058957e+04],
       [ 2.20000000e+00,  0.00000000e+00,  7.63704637e+03],
       ...,
       [ 1.80000000e+01,  9.00000000e+01, -1.81438277e-03],
       [ 1.90000000e+01,  9.00000000e+01, -1.31083599e-03],
       [ 2.00000000e+01,  9.00000000e+01, -9.63076223e-04]])

### Plots of 7 angles! 1D and 2D 